# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
browser = Browser('chrome')

DriverNotFoundError: Driver for chrome was not found.

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html=browser.html
soup=BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
table=soup.find('table', class_='table')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
# Loop through the scraped data to create a list of rows
header = []
rows = []
for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        header = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])  

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_weather_df = pd.DataFrame(rows, columns=header)


In [ ]:
# Confirm DataFrame was created successfully
mars_weather_df.head(50)

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_weather_df.dtypes

In [ ]:
# Change data types for data analysis
mars_weather_df['id']=mars_weather_df['id'].astype(int)
mars_weather_df['terrestrial_date']=mars_weather_df['terrestrial_date'].astype('datetime64')
mars_weather_df['sol']=mars_weather_df['sol'].astype(float)
mars_weather_df['ls']=mars_weather_df['ls'].astype(float)
mars_weather_df['month']=mars_weather_df['month'].astype(int)
mars_weather_df['min_temp']=mars_weather_df['min_temp'].astype(float)
mars_weather_df['pressure']=mars_weather_df['pressure'].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
mars_weather_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
len(mars_weather_df['month'].unique())

In [ ]:
# 2. How many Martian days' worth of data are there?
mars_weather_df['sol'].count()

In [ ]:
# 3. What is the average low temperature by month?
temp_by_month=mars_weather_df.groupby('month')['min_temp'].mean()

In [ ]:
# Plot the average temperature by month
temp_by_month.plot(label="Average Low Temperature")
plt.xlim(0,12)
plt.title('Temperature by month')
plt.ylabel('Mininium Temperature in Celsius')
plt.legend()
plt.show()

In [ ]:
temp_by_month = pd.DataFrame(temp_by_month).reset_index()
temp_by_month

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
cldest_month=temp_by_month.loc[ temp_by_month['min_temp'] == temp_by_month['min_temp'].min(), 'month' ].values[0]
hottest_month=temp_by_month.loc[ temp_by_month['min_temp'] == temp_by_month['min_temp'].max(), 'month' ].values[0]

cldest_month

In [ ]:
# 4. Average pressure by Martian month
average_pressure=mars_weather_df.groupby("month")['pressure'].mean()
average_pressure.head()

In [ ]:
# Plot the average pressure by month
pressure_by_month=mars_weather_df.groupby('month')['pressure'].mean()
pressure_by_month.plot(label="Average Pressure")
plt.xlim(0,12)
plt.title('Pressure by month')
plt.ylabel('The atmospheric pressure')
plt.legend()
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
plt.plot(mars_weather_df['min_temp'])

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
mars_weather_df.to_csv('mars_weather.csv')

In [ ]:
browser.quit()